# 페이지의 URL 정보 추출하기

In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re

In [12]:
# brew로 설치된 chromedriver의 path
path = "/Users/yanghaejun/homebrew/Caskroom/chromedriver/107.0.5304.62/chromedriver"

In [57]:
# 크롤링 할 사이트 주소를 정의
source_url = "https://m.sports.naver.com/qatar2022/news/index"

In [59]:
# 크롬 드라이버를 사용
# driver = webdriver.Chrome(path)
driver = webdriver.Chrome(path)

/var/folders/79/v97_0rqs3p34z9xx6y57xks80000gn/T/ipykernel_1831/2362576081.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [60]:
# 드라이버가 브라우징 할 페이지 소스를 입력
driver.get(source_url)
req = driver.page_source

In [89]:
# 사이트의 html 구조에 기반하여 데이터를 파싱
soup = BeautifulSoup(req, "html.parser")
content_div = soup.find("div", {"class": "News_main_section__1b-Va"})
content_uls = content_div.find("ul", {"class": "News_news_list__3_uUk"})
content_li = content_ul.find_all("li", {"class": "News_news_item__2prgi"})
print(content_li)

[]


In [55]:
# a 태그의 href 속성을 리스트로 추출하여 크롤링할 페이지 리스트를 생성
page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all("td")[0]
    td_url = first_td.find_all("a")
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get("href")
        if "png" not in page_url:
            page_urls.append(page_url)

In [51]:
# 중복 url을 제거
page_urls = list(set(page_urls))
for page in page_urls[:3]:
    print(page)

https://namu.wiki/w/%EC%99%80%EB%9D%BC%EC%BF%A0%20%EC%B9%98%EC%84%B8
https://namu.wiki/w/%ED%95%99%EA%B5%90%EA%B2%8C%EC%9E%84
https://namu.wiki/w/%EC%95%84%EC%82%B0%EC%8B%9C%20%EC%8B%9C%EB%82%B4%EB%B2%84%EC%8A%A4


In [39]:
# 하나의 '최근 변경된 문서'를 크롤링
driver.get(page_urls[0])
req = driver.page_source
soup = BeautifulSoup(req, "html.parser")
contents_table = soup.find(name = "theseed-dark-mode")
print(contents_table)

None


In [40]:
title = contents_table.find_all('h1')[0]
category = contents_table.find_all('ul')[0]
content_paragraphs = contents_table.find_all(name = "div", attr = {"class": "824bdff0"})
content_corpus_list = []

for paragraphs in content_paragraphs:
    content_corpus_list.append(paragraphs.titleext)
content_corpus = "".join(content_corpus_list)

print(title.text)
print()
print(category.text)
print()
print(content_corpus)

AttributeError: 'NoneType' object has no attribute 'find_all'